# 11. Hafta Lab-2: Malware Images
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
## İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
### Dr. Ferhat Özgür Çatak
Bu lab çalışması kapsamında Android zararlı yazılımlarının image haline getirilip sınıflandırma modeli oluşturulması anlatılacaktır.

In [1]:
from keras.models import Sequential
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split


from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import callbacks

from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


### Model
2 adet convolution katmanı ve tek katman sinir ağ modeli

In [2]:
data = []
labels = []

class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
        
        '''
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        '''
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(10, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("tanh"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same"))
        model.add(Activation("tanh"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(30))
        model.add(Activation("sigmoid"))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model
       

In [3]:
# image dosyalarını okuyarak numpy array'e dönüştür
f = open("results.txt","r")

In [4]:
iter = 0
for line in f:
    iter += 1
    if iter > 30:
        continue
    c,fname = line.split(",")
    fpath = "C:/Users/user/Documents/dataset/pngs/"+ fname.strip()
    
    print(fpath.strip())
    i = cv2.imread(fpath)
    image = cv2.resize(i, (2048,512))
    image = img_to_array(image)
    data.append(image)

    labels.append(c)

print(len(data))

C:/Users/user/Documents/dataset/pngs/d7de0ee80aa16beca37ccbbc30031995.png
C:/Users/user/Documents/dataset/pngs/7282c48bdad45f3861edd8244061c26e.png
C:/Users/user/Documents/dataset/pngs/e368fb1d80bbf24fdfb4ebae7806c885.png
C:/Users/user/Documents/dataset/pngs/f25e3352735aa210906527adf1140980.png
C:/Users/user/Documents/dataset/pngs/ee1bcb0d5036b4ba72036f79c538c8b1.png
C:/Users/user/Documents/dataset/pngs/9a031f2f5022fae13849b566a1b89579.png
C:/Users/user/Documents/dataset/pngs/6797aebf0ff789fbf37f543acc126a98.png
C:/Users/user/Documents/dataset/pngs/a14e9bfc6dfdfa6fca36a7aefe7590d1.png
C:/Users/user/Documents/dataset/pngs/d44cda7feb8e37d7373fbca2199c6820.png
C:/Users/user/Documents/dataset/pngs/20d4b9eb9377c499917c4d69bf4ccebe.png
C:/Users/user/Documents/dataset/pngs/43680d1914f28e14c90436e1d42984e2.png
C:/Users/user/Documents/dataset/pngs/acfce0995b14ff44eab18a90ba46c292.png
C:/Users/user/Documents/dataset/pngs/825fc8e2c5888e31d600a7b6ad3f0b57.png
C:/Users/user/Documents/dataset/pngs/9

In [5]:
data = np.array(data, dtype="float")
labels = np.array(labels)

print('data.shape',data.shape)
print('labels.shape',labels.shape)

data.shape (30, 512, 2048, 3)
labels.shape (30,)


### Modeli oluştur


In [6]:
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
dummy_Y = np_utils.to_categorical(encoded_Y)
y = encoded_Y
model = LeNet.build(width=2048, height=512, depth=3, classes=dummy_Y.shape[1])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 2048, 10)     760       
_________________________________________________________________
activation_1 (Activation)    (None, 512, 2048, 10)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 1024, 10)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 1024, 20)     5020      
_________________________________________________________________
activation_2 (Activation)    (None, 256, 1024, 20)     0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 512, 20)      0         
_________________________________________________________________
flat

### Modeli yükle

In [8]:
filepath="03-malware-detection.hdf5"

#model.load_weights(filepath)

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])


In [11]:
model.fit(data, dummy_Y, verbose=1, epochs=1, batch_size=5, validation_split=0.1 , callbacks=callbacks_list )

Train on 27 samples, validate on 3 samples
Epoch 1/1
27/27 [==============================] - 130s 5s/step - loss: 1.4507 - acc: 0.4074 - val_loss: 0.5164 - val_acc: 1.0000

Epoch 00001: loss improved from 1.62234 to 1.45070, saving model to 03-malware-detection.hdf5


### Model performansı

In [ ]:
y_hat = model.predict_classes(data)
cm = confusion_matrix(encoded_Y, y_hat)
print(cm)